In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from debug_utils import get_omodel, get_peft_ckpt, generate_outputs, adjust_input_apf
from datasets import load_dataset

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Start by checking stuff out for the webgpt model

In [59]:
with open('../basepath.txt', 'r') as file:
    bpath = file.read().strip()

with open('../apfpath.txt', 'r') as file:
    apfpath = file.read().strip()
    
basename = apfpath+"apfarm_models/sft10k/"
origmodel, toker = get_omodel(basename)

Loading checkpoint shards:   0%|                                                                                                                                         | 0/3 [00:02<?, ?it/s]

KeyboardInterrupt



In [40]:
generation_kwargs = {
    "min_new_tokens": -1,
    "max_new_tokens":512,
    #"top_k": 0.0,
    "top_p": 1,
    #"temperature": 0.9,
    "do_sample": True,
    #"repetition_penalty": 1.2,
    #"pad_token_id": tokenizer.pad_token_id,
    #"eos_token_id": tokenizer.eos_token_id,
}

In [10]:
inps = load_dataset("openai/webgpt_comparisons", split="train")

Found cached dataset webgpt_comparisons (/home/prasann/.cache/huggingface/datasets/openai___webgpt_comparisons/default/0.0.0/8b5d5879cdc98c4c0099af6053dffe8d504588d43d3b11f1b1ec223ab1e8db0a)


In [53]:
ind = 20
qtemp = adjust_input_apf(inps[ind]['question']['full_text'])
toks, outs = generate_outputs([qtemp], toker, origmodel, generation_kwargs)
print(len(toks[0]))
print(outs[0])

<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
How many volunteer hours were estimated to have supported the 2012 Summer Olympics?

### Response:Approximately 70,000 volunteer hours are estimated to have been needed to support the 2012 Summer Olympics. This is equivalent to about 280 volunteers working full-time for one year. 70,000 volunteers were recruited for the Games, and they were divided into four categories: Gamesmaker, Venue Volunteer, Event Volunteer and Media Volunteer. The Gamesmakers, Venue Volunteers, and Event Volunteers were responsible for helping with various activities during the Games such as providing information to spectators, controlling access to venues, and manning various events. Media Volunteers were responsible for assisting with broadcasting, media operations, and photography. Volunteers were required to attend training sessions and adhere to a code of conduct, and they were gi

In [ ]:
#SFT Starts out reasonably short

In [60]:
adaptpath = bpath+"/webgpt-llama/checkpoints/wgptapsft/step_125"
amod = get_peft_ckpt(origmodel, adaptpath)

In [62]:
ind = 40
qtemp = adjust_input_apf(inps[ind]['question']['full_text'])
toks, outs = generate_outputs([qtemp], toker, amod, generation_kwargs)
print(len(toks[0]))
print(outs[0])

330
<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Name the 1980's hit sung by Tina Turner and Rod Stewart?

### Response:The 1980's hit sung by Tina Turner and Rod Stewart is "Simply the Best". It was written and composed by Mike Chapman and Holly Knight, and released as a single from Tina Turner's 1984 album, Break Every Rule. The song was also included as a track on Rod Stewart's album Outshined in the same year. The single reached number one on the US Billboard Hot 100 chart and on the UK Singles Chart, respectively, becoming one of Turner's signature songs. It was also nominated for the Grammy Award for Best Female Rock Vocal Performance in 1985. The song is considered to be one of the greatest songs of the 80's and is often seen as Turner's signature song. Turner also performs the song at her concerts, and it is one of the songs Stewart most frequently performs in his shows. Despite its success, the 